# Predictions for high-loop-density areas

Author: Amulya Garimella

Last updated: 2025/05/02

In [ ]:
import cooler
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from scipy.sparse import coo_matrix
import pybedtools
import pandas as pd
import sklearn.metrics
import scipy.stats
import torch
import joblib

## Input files

In [11]:
input_cool_file = "/Users/amulyagarimella/Documents/2241finalproject/data/GM12878.GSE115524/GM12878.GSE115524.Homo_Sapiens.CTCF.b1.mcool"

In [39]:
input_cool_file_downsampled_chr1 = "/Users/amulyagarimella/Documents/2241finalproject/data/GM12878.GSE115524/processed/GM12878.GSE115524.Homo_Sapiens.CTCF.b1.ds.16.chr1.cool::/"

input_cool_file_downsampled_chr21 = "/Users/amulyagarimella/Documents/2241finalproject/data/GM12878.GSE115524/processed/GM12878.GSE115524.Homo_Sapiens.CTCF.b1.ds.16.chr21.cool::/"


In [40]:
input_loops_file = f"/Users/amulyagarimella/Documents/2241finalproject/data/GM12878.GSE115524/loops/GM12878.GSE115524.Homo_Sapiens.CTCF.b1.{input_res}.fithichip_hp.longrange.bed"

def process_loop_catalog_dataset(dataset_path):
    """
    Process the loop catalog dataset to extract relevant columns and convert them to a PyRanges object.
    """
    df = pd.read_csv(dataset_path, header=None, sep='\t')
    extra = df[3].str.split(r',|:|-', expand=True)
    df = pd.concat([df.drop(columns=[3]), extra], axis=1)
    df.columns = ["chr_1", "start_1", "end_1", "chr_2", "start_2", "end_2", "score"]
    # Convert start and end columns to integers
    df[["start_1", "end_1", "start_2", "end_2"]] = df[["start_1", "end_1", "start_2", "end_2"]].astype(int)
    
    # Convert score to float
    df["score"] = df["score"].astype(float)
    return df

loops = process_loop_catalog_dataset(input_loops_file)

In [47]:
# extract chrom sizes and coolers

chrsizes = pd.read_csv("/Users/amulyagarimella/Documents/2241finalproject/HiC2MicroC/data/hg38.sizes", sep="\t", header=None, index_col=0)
input_res = 10000

clr = cooler.Cooler(f'{input_cool_file}::/resolutions/{input_res}')
clr_downsampled_chr1 = cooler.Cooler(input_cool_file_downsampled_chr1)
clr_downsampled_chr21 = cooler.Cooler(input_cool_file_downsampled_chr21)

## Feature and label extraction

In [48]:
# Feature extraction: divide mat into submatrices
# Labels: determine number of loops in each submatrix

def get_features_and_labels (input_clr, input_loops, chrnames, res, submatrix_bins):
    features = []
    labels = []

    for chrname in chrnames:
        chrlen = chrsizes.loc[chrname].values[0]
        chrlen_bins = chrlen//res
        min_submatrices = 100
        submatrix_len = submatrix_bins*res
        submatrix_len_bins = submatrix_len//res

        overlap_len = max((submatrix_len_bins*min_submatrices - chrlen_bins) // (min_submatrices - 1),0)
        stridelen_bins = submatrix_len_bins - overlap_len

        for i in np.arange(0, chrlen_bins, stridelen_bins):
            # Extract the submatrix
            i_end = min(i + submatrix_len_bins, chrlen_bins)
            #print(f"Bin {i} to {i_end}")
            
            start_coord = int(i*res)
            end_coord = int(i_end*res)
            #print(f"Start coord: {start_coord}, End coord: {end_coord}")

            counts = input_clr.matrix(sparse=True, balance=False).fetch(
                f"{chrname}:{start_coord}-{end_coord}"
            )

            try:
                counts.set_shape((1, submatrix_len_bins ** 2))
            except Exception as e:
                print(f"Error setting shape for submatrix {i}: {e}")
                counts = scipy.sparse.coo_matrix((counts.data, (counts.row, counts.col)), shape=(submatrix_len_bins, submatrix_len_bins))
                counts.set_shape((1, submatrix_len_bins ** 2))
                    
            features.append(counts)
            # Determine the number of loops in the submatrix
            loops_in_submat = input_loops.loc[
                (input_loops['chr_1'] == chrname) & 
                (input_loops['start_1'] >= start_coord) & 
                (input_loops['end_1'] <= end_coord) & 
                (input_loops['chr_2'] == chrname) & 
                (input_loops['start_2'] >= start_coord) & 
                (input_loops['end_2'] <= end_coord)
            ]

            labels.append(len(loops_in_submat))
            if len(loops_in_submat) > 0:
                print(f"Submatrix {i} has {len(loops_in_submat)} loops")
            else:
                print(f"Submatrix {i} has no loops")

    # Convert to numpy arrays
    features = np.array(features)
    print(f"Created {len(features)} submatrices")
    print(f"Feature shape: {features.shape}")

    # Convert labels to numpy array
    labels = np.array(labels)
    print(f"Labels shape: {labels.shape}")

    features_coo = scipy.sparse.vstack(features, format='coo')
    features_csr = scipy.sparse.vstack([x.tocsr() for x in features], format='csr')
    features_csc = scipy.sparse.vstack([x.tocsc() for x in features], format='csc')
    

    return features_coo, features_csr, features_csc, labels

In [49]:
chrnames = chrsizes.index.values

In [50]:
test_chrname = "chr21"
train_chrnames = [x for x in chrnames if x != test_chrname]

## feature extraction

### full

In [51]:
feats_train_coo, feats_train_csr, feats_train_csc, labels_train = get_features_and_labels(clr, loops, ["chr1"] ,input_res,40)

Submatrix 0 has no loops
Submatrix 40 has no loops
Submatrix 80 has 16 loops
Submatrix 120 has 30 loops
Submatrix 160 has 14 loops
Submatrix 200 has 12 loops
Submatrix 240 has 40 loops
Submatrix 280 has no loops
Submatrix 320 has no loops
Submatrix 360 has 10 loops
Submatrix 400 has no loops
Submatrix 440 has no loops
Submatrix 480 has no loops
Submatrix 520 has no loops
Submatrix 560 has no loops
Submatrix 600 has 14 loops
Submatrix 640 has 4 loops
Submatrix 680 has no loops
Submatrix 720 has no loops
Submatrix 760 has 8 loops
Submatrix 800 has 26 loops
Submatrix 840 has 8 loops
Submatrix 880 has 22 loops
Submatrix 920 has 28 loops
Submatrix 960 has 16 loops
Submatrix 1000 has 4 loops
Submatrix 1040 has 10 loops
Submatrix 1080 has no loops
Submatrix 1120 has no loops
Submatrix 1160 has 10 loops
Submatrix 1200 has 18 loops
Submatrix 1240 has 2 loops
Submatrix 1280 has no loops
Submatrix 1320 has 2 loops
Submatrix 1360 has 8 loops
Submatrix 1400 has no loops
Submatrix 1440 has 2 loops
S

In [52]:
feats_test_coo, feats_test_csc, feats_test_csr, labels_test = get_features_and_labels(clr, loops,[test_chrname], input_res,40)

Submatrix 0 has no loops
Submatrix 40 has no loops
Submatrix 80 has no loops
Submatrix 120 has no loops
Submatrix 160 has no loops
Submatrix 200 has no loops
Submatrix 240 has no loops
Submatrix 280 has no loops
Submatrix 320 has no loops
Submatrix 360 has no loops
Submatrix 400 has no loops
Submatrix 440 has no loops
Submatrix 480 has 2 loops
Submatrix 520 has no loops
Submatrix 560 has no loops
Submatrix 600 has no loops
Submatrix 640 has no loops
Submatrix 680 has no loops
Submatrix 720 has no loops
Submatrix 760 has no loops
Submatrix 800 has no loops
Submatrix 840 has 2 loops
Submatrix 880 has 4 loops
Submatrix 920 has no loops
Submatrix 960 has no loops
Submatrix 1000 has no loops
Submatrix 1040 has no loops
Submatrix 1080 has no loops
Submatrix 1120 has no loops
Submatrix 1160 has no loops
Submatrix 1200 has no loops
Submatrix 1240 has no loops
Submatrix 1280 has no loops
Submatrix 1320 has no loops
Submatrix 1360 has no loops
Submatrix 1400 has 2 loops
Submatrix 1440 has 24 loo

In [53]:
loop_threshold = 0

binary_labels_train = np.where(labels_train > np.quantile(labels_train, loop_threshold), 1, 0)

binary_labels_test = np.where(labels_test > np.quantile(labels_test, loop_threshold), 1, 0)


### downsampled

In [57]:
feats_train_coo_16down, feats_train_csr_16down, feats_train_csc_16down, labels_train_16down = get_features_and_labels(clr_downsampled_chr1, loops, ["chr1"] ,input_res,40)

Submatrix 0 has no loops
Submatrix 40 has no loops
Submatrix 80 has 16 loops
Submatrix 120 has 30 loops
Submatrix 160 has 14 loops
Submatrix 200 has 12 loops
Submatrix 240 has 40 loops
Submatrix 280 has no loops
Submatrix 320 has no loops
Submatrix 360 has 10 loops
Submatrix 400 has no loops
Submatrix 440 has no loops
Submatrix 480 has no loops
Submatrix 520 has no loops
Submatrix 560 has no loops
Submatrix 600 has 14 loops
Submatrix 640 has 4 loops
Submatrix 680 has no loops
Submatrix 720 has no loops
Submatrix 760 has 8 loops
Submatrix 800 has 26 loops
Submatrix 840 has 8 loops
Submatrix 880 has 22 loops
Submatrix 920 has 28 loops
Submatrix 960 has 16 loops
Submatrix 1000 has 4 loops
Submatrix 1040 has 10 loops
Submatrix 1080 has no loops
Submatrix 1120 has no loops
Submatrix 1160 has 10 loops
Submatrix 1200 has 18 loops
Submatrix 1240 has 2 loops
Submatrix 1280 has no loops
Submatrix 1320 has 2 loops
Submatrix 1360 has 8 loops
Submatrix 1400 has no loops
Submatrix 1440 has 2 loops
S

In [58]:
feats_test_coo_16down, feats_test_csr_16down, feats_test_csc_16down, labels_test_16down = get_features_and_labels(clr_downsampled_chr21, loops, ["chr21"] ,input_res,40)

Submatrix 0 has no loops
Submatrix 40 has no loops
Submatrix 80 has no loops
Submatrix 120 has no loops
Submatrix 160 has no loops
Submatrix 200 has no loops
Submatrix 240 has no loops
Submatrix 280 has no loops
Submatrix 320 has no loops
Submatrix 360 has no loops
Submatrix 400 has no loops
Submatrix 440 has no loops
Submatrix 480 has 2 loops
Submatrix 520 has no loops
Submatrix 560 has no loops
Submatrix 600 has no loops
Submatrix 640 has no loops
Submatrix 680 has no loops
Submatrix 720 has no loops
Submatrix 760 has no loops
Submatrix 800 has no loops
Submatrix 840 has 2 loops
Submatrix 880 has 4 loops
Submatrix 920 has no loops
Submatrix 960 has no loops
Submatrix 1000 has no loops
Submatrix 1040 has no loops
Submatrix 1080 has no loops
Submatrix 1120 has no loops
Submatrix 1160 has no loops
Submatrix 1200 has no loops
Submatrix 1240 has no loops
Submatrix 1280 has no loops
Submatrix 1320 has no loops
Submatrix 1360 has no loops
Submatrix 1400 has 2 loops
Submatrix 1440 has 24 loo

In [59]:
loop_threshold = 0

binary_labels_train_16down = np.where(labels_train_16down > np.quantile(labels_train_16down, loop_threshold), 1, 0)

binary_labels_test_16down = np.where(labels_test_16down > np.quantile(labels_test_16down, loop_threshold), 1, 0)


## baseline 1: regressions

### Ridge

#### full 

In [ ]:
model = Ridge(
    alpha=1.0,
    solver="lsqr",          # or 'lsqr','sparse_cg','lbfgs'
    max_iter=1000,
    random_state=42
)


In [8]:
model.fit(feats_train_csc, labels_train)       # accepts sparse X when using supported solvers :contentReference[oaicite:7]{index=7}
coefs = model.coef_

NameError: name 'feats_train_csc' is not defined

In [ ]:
predictions_test = model.predict(feats_test_csc)
clipped_predictions_test = np.clip(predictions_test, 0, None)

print("MSE without clipping")
print(sklearn.metrics.mean_squared_error(labels_test, predictions_test))
print("MSE with clipping")
print(sklearn.metrics.mean_squared_error(labels_test, clipped_predictions_test))

print("Pearson without clipping")
print(scipy.stats.pearsonr(labels_test, predictions_test))
print("Pearson with clipping")
print(scipy.stats.pearsonr(labels_test, clipped_predictions_test))

print("Spearman without clipping")
print(scipy.stats.spearmanr(labels_test, predictions_test))
print("Spearman with clipping")
print(scipy.stats.spearmanr(labels_test, clipped_predictions_test))

142.81760129880072
121.26894194249316


In [ ]:
ridge_model_filename = '../prediction_models/ridge_model_csc_40.joblib'
joblib.dump(model, ridge_model_filename)

['ridge_model_csc_40.joblib']

### downsampled

In [66]:
model_16down = Ridge(
    alpha=1.0,
    solver="lsqr", # or 'sparse_cg','lbfgs'
    max_iter=1000,
    random_state=42
)


In [68]:
model_16down.fit(feats_train_csc_16down, labels_train_16down)
coefs_16down = model_16down.coef_

In [71]:
predictions_test_16down = model_16down.predict(feats_test_csc_16down)
clipped_predictions_test_16down = np.clip(predictions_test_16down, 0, None)

print("MSE without clipping")
print(sklearn.metrics.mean_squared_error(labels_test_16down, predictions_test_16down))
print("MSE with clipping")
print(sklearn.metrics.mean_squared_error(labels_test_16down, clipped_predictions_test_16down))

print("Pearson without clipping")
print(scipy.stats.pearsonr(labels_test_16down, predictions_test_16down))
print("Pearson with clipping")
print(scipy.stats.pearsonr(labels_test_16down, clipped_predictions_test_16down))

print("Spearman without clipping")
print(scipy.stats.spearmanr(labels_test_16down, predictions_test_16down))
print("Spearman with clipping")
print(scipy.stats.spearmanr(labels_test_16down,clipped_predictions_test_16down))

MSE without clipping
168.88034632053905
MSE with clipping
165.13893785439916
Pearson without clipping
PearsonRResult(statistic=np.float64(0.41724346004876645), pvalue=np.float64(2.8687810459764013e-06))
Pearson with clipping
PearsonRResult(statistic=np.float64(0.4333282258548408), pvalue=np.float64(1.0603825273053879e-06))
Spearman without clipping
SignificanceResult(statistic=np.float64(0.4987692957246943), pvalue=np.float64(1.0449333861452937e-08))
Spearman with clipping
SignificanceResult(statistic=np.float64(0.5453782917523493), pvalue=np.float64(2.04157092968078e-10))


In [73]:
ridge_model_16down_filename = '../prediction_models/ridge_model_csc_40_16down.joblib'
joblib.dump(model_16down, ridge_model_16down_filename)

['../prediction_models/ridge_model_csc_40_16down.joblib']

## baseline 2: logreg

In [35]:
logreg = LogisticRegression(max_iter=10000, random_state=42, penalty='l2')
logreg.fit(feats_train_csr, binary_labels_train)

LogisticRegression(max_iter=10000, random_state=42)

In [36]:
binary_labels_test = np.where(labels_test > np.quantile(labels_test, loop_threshold), 1, 0)

binary_predictions_test = logreg.predict(feats_test_csr)

binary_predictions_test_proba = logreg.predict_proba(feats_test_csr)[:, 1]
error = sklearn.metrics.mean_squared_error(binary_labels_test, binary_predictions_test)

print(f"Binary classification accuracy: {sklearn.metrics.accuracy_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification balanced accuracy: {sklearn.metrics.balanced_accuracy_score(binary_labels_test, binary_predictions_test)}")

print(f"Binary classification precision: {sklearn.metrics.precision_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification recall: {sklearn.metrics.recall_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification f1 score: {sklearn.metrics.f1_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification roc_auc: {sklearn.metrics.roc_auc_score(binary_labels_test, binary_predictions_test_proba)}")
print(f"Binary classification roc_auc (labels): {sklearn.metrics.roc_auc_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification roc_auc (predictions): {sklearn.metrics.roc_auc_score(binary_labels_test, binary_predictions_test)}")
print(f"Binary classification roc_auc (predictions probabilities): {sklearn.metrics.roc_auc_score(binary_labels_test, binary_predictions_test_proba)}")

Binary classification accuracy: 0.794392523364486
Binary classification balanced accuracy: 0.8932038834951457
Binary classification precision: 1.0
Binary classification recall: 0.7864077669902912
Binary classification f1 score: 0.8804347826086957
Binary classification roc_auc: 0.9830097087378641
Binary classification roc_auc (labels): 0.8932038834951457
Binary classification roc_auc (predictions): 0.8932038834951457
Binary classification roc_auc (predictions probabilities): 0.9830097087378641


In [37]:
baseline = sum(binary_labels_test) / len(binary_labels_test)
print(max(baseline, 1 - baseline))

0.9626168224299065


In [38]:
binary_predictions_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0])